In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq 

In [ ]:
!sudo add-apt-repository ppa:webupd8team/java
!sudo apt-get update
!sudo apt-get install oracle-java8-installer

 The Oracle JDK License has changed for releases starting April 16, 2019.

The new Oracle Technology Network License Agreement for Oracle Java SE is substantially different from prior Oracle JDK licenses. The new license permits certain uses, such as personal use and development use, at no cost -- but other uses authorized under prior Oracle JDK licenses may no longer be available. Please review the terms carefully before downloading and using this product. An FAQ is available here: https://www.oracle.com/technetwork/java/javase/overview/oracle-jdk-faqs.html

Oracle Java downloads now require logging in to an Oracle account to download Java updates, like the latest Oracle Java 8u211 / Java SE 8u212. Because of this I cannot update the PPA with the latest Java (and the old links were broken by Oracle).

For this reason, THIS PPA IS DISCONTINUED.

UPDATE:

For Oracle Java 17, see a different PPA -> https://www.linuxuprising.com/2021/09/how-to-install-oracle-java-17-lts-on.html

Old descr

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/CICMalMem2022_Malware_Classification/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Proje


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
      .master("local") \
      .appName("Proje") \
      .config("spark.executor.memory", "12gb") \
      .getOrCreate()
    
sc = spark.sparkContext

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sbn
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras

In [ ]:
dataset = pd.read_csv("Obfuscated-MalMem2022.csv")

In [ ]:
dataset.tail(5)

,Category,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,...,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class
58591,Ransomware-Shade-fa03be3078d1b9840f06745f160eb...,37,15,10.108108,0,215.486487,1453,39.270270,7973,215.486487,...,221,26,24,116,0,120,86,0,8,Malware
58592,Ransomware-Shade-f56687137caf9a67678cde91e4614...,37,14,9.945946,0,190.216216,1347,36.405405,7038,190.216216,...,221,26,24,116,0,116,88,0,8,Malware
58593,Ransomware-Shade-faddeea111a25da4d0888f3044ae9...,38,15,9.842105,0,210.026316,1448,38.105263,7982,215.729730,...,221,26,24,116,0,120,88,0,8,Malware
58594,Ransomware-Shade-f866c086af2e1d8ebaa6f2c863157...,37,15,10.243243,0,215.513513,1452,39.243243,7974,215.513513,...,221,26,24,116,0,120,87,0,8,Malware
58595,Ransomware-Shade-955d9af38346c1755527bd196668e...,38,15,9.868421,0,213.026316,1487,39.131579,8095,213.026316,...,221,26,24,116,0,120,86,0,8,Malware


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58596 entries, 0 to 58595
Data columns (total 57 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Category                                58596 non-null  object 
 1   pslist.nproc                            58596 non-null  int64  
 2   pslist.nppid                            58596 non-null  int64  
 3   pslist.avg_threads                      58596 non-null  float64
 4   pslist.nprocs64bit                      58596 non-null  int64  
 5   pslist.avg_handlers                     58596 non-null  float64
 6   dlllist.ndlls                           58596 non-null  int64  
 7   dlllist.avg_dlls_per_proc               58596 non-null  float64
 8   handles.nhandles                        58596 non-null  int64  
 9   handles.avg_handles_per_proc            58596 non-null  float64
 10  handles.nport                           58596 non-null  in

In [ ]:
dataset.describe()

,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,handles.nport,...,svcscan.nservices,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric
count,58596.000000,58596.000000,58596.000000,58596.0,58596.000000,58596.000000,58596.000000,5.859600e+04,58596.000000,58596.0,...,58596.000000,58596.000000,58596.000000,58596.000000,58596.000000,58596.0,58596.000000,58596.000000,58596.000000,58596.000000
mean,41.394771,14.713837,11.341655,0.0,247.509819,1810.805447,43.707806,1.025858e+04,249.560958,0.0,...,391.347549,221.406581,25.996245,25.063417,116.879514,0.0,121.995546,86.905659,0.000853,7.999881
std,5.777249,2.656748,1.588231,0.0,111.857790,329.782639,5.742023,4.866864e+03,145.999866,0.0,...,4.529704,1.991087,0.170790,1.529628,1.550401,0.0,2.822858,3.134117,0.029199,0.010929
min,21.000000,8.000000,1.650000,0.0,34.962500,670.000000,7.333333,3.514000e+03,71.139241,0.0,...,94.000000,55.000000,6.000000,7.000000,26.000000,0.0,30.000000,50.000000,0.000000,7.000000
25%,40.000000,12.000000,9.972973,0.0,208.725000,1556.000000,38.833333,8.393000e+03,209.648228,0.0,...,389.000000,221.000000,26.000000,24.000000,116.000000,0.0,121.000000,87.000000,0.000000,8.000000
50%,41.000000,15.000000,11.000000,0.0,243.963710,1735.000000,42.781524,9.287500e+03,247.208951,0.0,...,389.000000,221.000000,26.000000,24.000000,116.000000,0.0,122.000000,87.000000,0.000000,8.000000
75%,43.000000,16.000000,12.861955,0.0,289.974322,2087.000000,49.605280,1.219300e+04,291.355050,0.0,...,395.000000,222.000000,26.000000,27.000000,118.000000,0.0,123.000000,88.000000,0.000000,8.000000
max,240.000000,72.000000,16.818182,0.0,24845.951220,3443.000000,53.170732,1.047310e+06,33784.193550,0.0,...,395.000000,222.000000,26.000000,27.000000,118.000000,0.0,129.000000,89.000000,1.000000,8.000000


In [ ]:
dataset.Class.value_counts()

Benign     29298
Malware    29298
Name: Class, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

dataset["Class"]= LabelEncoder().fit_transform(dataset["Class"])

In [ ]:
dataset.Class.value_counts()

0    29298
1    29298
Name: Class, dtype: int64

In [ ]:
dataset.value_counts("Category")

Category
Benign                                                                                         29298
Spyware-Transponder-0dcb10bb8760428a704587c3976c5c045315cca59333f6f31eb1690f21df8553-6.raw         2
Spyware-Transponder-0dcddfad933171346232bd3d5f1186d66126b0e0a0e15385856e9e15c2c02e15-5.raw         2
Spyware-Transponder-0dcddfad933171346232bd3d5f1186d66126b0e0a0e15385856e9e15c2c02e15-4.raw         2
Spyware-Transponder-0dcddfad933171346232bd3d5f1186d66126b0e0a0e15385856e9e15c2c02e15-3.raw         2
                                                                                               ...  
Spyware-180solutions-0a09c38c3c5003d5adff784e1337c3207be62fafbebde86bff2a1dbe65a9fb42-1.raw        1
Spyware-180solutions-01b2e72a6ca18b91a382a67099d61045e167f24da53470478110ade44180186e-9.raw        1
Spyware-180solutions-01b2e72a6ca18b91a382a67099d61045e167f24da53470478110ade44180186e-8.raw        1
Spyware-180solutions-01b2e72a6ca18b91a382a67099d61045e167f24da53470478110ade441801

In [ ]:
dataset = dataset.iloc[:,1:]

In [ ]:
zero= []
dataset.to_numpy()
for i in dataset:
  if dataset[i].sum(axis=0) == 0:
     zero.append(i)
print(zero)

['pslist.nprocs64bit', 'handles.nport', 'svcscan.interactive_process_services']


In [ ]:
dataset= dataset.drop(columns= ['pslist.nprocs64bit', 
                                'handles.nport', 'svcscan.interactive_process_services'])

In [ ]:
dataset.head(3)

,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,handles.nfile,handles.nevent,...,svcscan.nservices,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class
0,45,17,10.555556,202.844444,1694,38.50000,9129,212.302326,670,3161,...,389,221,26,24,116,121,87,0,8,0
1,47,19,11.531915,242.234043,2074,44.12766,11385,242.234043,840,3761,...,392,222,26,24,118,122,87,0,8,0
2,40,14,14.725000,288.225000,1932,48.30000,11529,288.225000,1050,3996,...,395,222,26,27,118,120,88,0,8,0


In [ ]:
dataset = spark.createDataFrame(dataset)

In [ ]:
dataset = dataset.toDF(*(c.replace('.', '_') for c in dataset.columns))

In [ ]:
dataset.printSchema()

root
 |-- pslist_nproc: long (nullable = true)
 |-- pslist_nppid: long (nullable = true)
 |-- pslist_avg_threads: double (nullable = true)
 |-- pslist_avg_handlers: double (nullable = true)
 |-- dlllist_ndlls: long (nullable = true)
 |-- dlllist_avg_dlls_per_proc: double (nullable = true)
 |-- handles_nhandles: long (nullable = true)
 |-- handles_avg_handles_per_proc: double (nullable = true)
 |-- handles_nfile: long (nullable = true)
 |-- handles_nevent: long (nullable = true)
 |-- handles_ndesktop: long (nullable = true)
 |-- handles_nkey: long (nullable = true)
 |-- handles_nthread: long (nullable = true)
 |-- handles_ndirectory: long (nullable = true)
 |-- handles_nsemaphore: long (nullable = true)
 |-- handles_ntimer: long (nullable = true)
 |-- handles_nsection: long (nullable = true)
 |-- handles_nmutant: long (nullable = true)
 |-- ldrmodules_not_in_load: long (nullable = true)
 |-- ldrmodules_not_in_init: long (nullable = true)
 |-- ldrmodules_not_in_mem: long (nullable = true

In [ ]:
dataset.drop("Class").toPandas().columns

Index(['pslist_nproc', 'pslist_nppid', 'pslist_avg_threads',
       'pslist_avg_handlers', 'dlllist_ndlls', 'dlllist_avg_dlls_per_proc',
       'handles_nhandles', 'handles_avg_handles_per_proc', 'handles_nfile',
       'handles_nevent', 'handles_ndesktop', 'handles_nkey', 'handles_nthread',
       'handles_ndirectory', 'handles_nsemaphore', 'handles_ntimer',
       'handles_nsection', 'handles_nmutant', 'ldrmodules_not_in_load',
       'ldrmodules_not_in_init', 'ldrmodules_not_in_mem',
       'ldrmodules_not_in_load_avg', 'ldrmodules_not_in_init_avg',
       'ldrmodules_not_in_mem_avg', 'malfind_ninjections',
       'malfind_commitCharge', 'malfind_protection',
       'malfind_uniqueInjections', 'psxview_not_in_pslist',
       'psxview_not_in_eprocess_pool', 'psxview_not_in_ethread_pool',
       'psxview_not_in_pspcid_list', 'psxview_not_in_csrss_handles',
       'psxview_not_in_session', 'psxview_not_in_deskthrd',
       'psxview_not_in_pslist_false_avg',
       'psxview_not_in_eproc

In [ ]:
from pyspark.ml.feature import VectorAssembler

assemblerInput = ['pslist_nproc', 'pslist_nppid', 'pslist_avg_threads',
       'pslist_avg_handlers', 'dlllist_ndlls', 'dlllist_avg_dlls_per_proc',
       'handles_nhandles', 'handles_avg_handles_per_proc', 'handles_nfile',
       'handles_nevent', 'handles_ndesktop', 'handles_nkey', 'handles_nthread',
       'handles_ndirectory', 'handles_nsemaphore', 'handles_ntimer',
       'handles_nsection', 'handles_nmutant', 'ldrmodules_not_in_load',
       'ldrmodules_not_in_init', 'ldrmodules_not_in_mem',
       'ldrmodules_not_in_load_avg', 'ldrmodules_not_in_init_avg',
       'ldrmodules_not_in_mem_avg', 'malfind_ninjections',
       'malfind_commitCharge', 'malfind_protection',
       'malfind_uniqueInjections', 'psxview_not_in_pslist',
       'psxview_not_in_eprocess_pool', 'psxview_not_in_ethread_pool',
       'psxview_not_in_pspcid_list', 'psxview_not_in_csrss_handles',
       'psxview_not_in_session', 'psxview_not_in_deskthrd',
       'psxview_not_in_pslist_false_avg',
       'psxview_not_in_eprocess_pool_false_avg',
       'psxview_not_in_ethread_pool_false_avg',
       'psxview_not_in_pspcid_list_false_avg',
       'psxview_not_in_csrss_handles_false_avg',
       'psxview_not_in_session_false_avg', 'psxview_not_in_deskthrd_false_avg',
       'modules_nmodules', 'svcscan_nservices', 'svcscan_kernel_drivers',
       'svcscan_fs_drivers', 'svcscan_process_services',
       'svcscan_shared_process_services', 'svcscan_nactive',
       'callbacks_ncallbacks', 'callbacks_nanonymous', 'callbacks_ngeneric']

assembler = VectorAssembler(inputCols= assemblerInput,  outputCol='vectorized_features')

vectorized_dataset = assembler.transform(dataset)

In [ ]:
from pyspark.ml.feature import Normalizer

normalized_dataset=Normalizer().setP(1).setInputCol("vectorized_features").setOutputCol("features")
normalized_data=normalized_dataset.transform(vectorized_dataset)

In [ ]:
dataset=normalized_data.select('features', 'Class')

In [ ]:
data = dataset.toPandas()

In [ ]:
data.head(5)

,features,Class
0,"[0.0022975428262595704, 0.0008679606232536155,...",0
1,"[0.001949461062095377, 0.0007880800038257907, ...",0
2,"[0.0016497500064885586, 0.0005774125022709955,...",0
3,"[0.0017672059900451793, 0.0007179274334558541,...",0
4,"[0.0016734215373824654, 0.0006374939190028439,...",0


In [ ]:
data.to_csv("preprocessed_dataset.csv")